In [1]:
import numpy as np
import time
import copy

from vgc.datatypes.Objects import PkmTeam, Pkm, GameState, Weather
from vgc.engine.PkmBattleEnv import PkmBattleEnv
from vgc.util.generator.PkmTeamGenerators import RandomTeamGenerator

from vgc.datatypes.Constants import TYPE_CHART_MULTIPLIER, MAX_HIT_POINTS, MOVE_MAX_PP, DEFAULT_TEAM_SIZE
from vgc.datatypes.Objects import PkmMove, Pkm, PkmTeam, GameState, Weather
from vgc.datatypes.Types import PkmStat, PkmType, WeatherCondition, \
    N_TYPES, N_STATUS, N_STATS, N_ENTRY_HAZARD, N_WEATHER, PkmStatus, PkmEntryHazard

import math
import pprint
import pickle

### Confirm dmg estimate matches reality

In [2]:
'''
reviewed code, looks teh same

	https://gitlab.com/DracoStriker/pokemon-vgc-engine/-/blob/master/vgc/engine/PkmBattleEnv.py?ref_type=heads#L489

	fixed_damage = self.__get_fixed_damage()
        if fixed_damage > 0. and TYPE_CHART_MULTIPLIER[move.type][opp_pkm.type] > 0.:
            damage = fixed_damage
        else:
            stab = 1.5 if move.type == pkm.type else 1.
            if (move.type == PkmType.WATER and self.weather.condition == WeatherCondition.RAIN) or (
                    move.type == PkmType.FIRE and self.weather.condition == WeatherCondition.SUNNY):
                weather = 1.5
            elif (move.type == PkmType.WATER and self.weather.condition == WeatherCondition.SUNNY) or (
                    move.type == PkmType.FIRE and self.weather.condition == WeatherCondition.RAIN):
                weather = .5
            else:
                weather = 1.
            stage_level = team.stage[PkmStat.ATTACK] - opp_team.stage[PkmStat.DEFENSE]
            stage = (stage_level + 2.) / 2 if stage_level >= 0. else 2. / (np.abs(stage_level) + 2.)
            multiplier = TYPE_CHART_MULTIPLIER[move.type][opp_pkm.type] if move != Struggle else 1.0
            damage = multiplier * stab * weather * stage * move.power

    def __get_fixed_damage(self) -> float:
        damage = self.move_view.damage
        self.move_view._damage = 0.
        return damage

    missing the struggle part

Code testing
    add debug to estiamte dmg part
    after found a difference see what is happening in the code

'''

'\nreviewed code, looks teh same\n\n\thttps://gitlab.com/DracoStriker/pokemon-vgc-engine/-/blob/master/vgc/engine/PkmBattleEnv.py?ref_type=heads#L489\n\n\tfixed_damage = self.__get_fixed_damage()\n        if fixed_damage > 0. and TYPE_CHART_MULTIPLIER[move.type][opp_pkm.type] > 0.:\n            damage = fixed_damage\n        else:\n            stab = 1.5 if move.type == pkm.type else 1.\n            if (move.type == PkmType.WATER and self.weather.condition == WeatherCondition.RAIN) or (\n                    move.type == PkmType.FIRE and self.weather.condition == WeatherCondition.SUNNY):\n                weather = 1.5\n            elif (move.type == PkmType.WATER and self.weather.condition == WeatherCondition.SUNNY) or (\n                    move.type == PkmType.FIRE and self.weather.condition == WeatherCondition.RAIN):\n                weather = .5\n            else:\n                weather = 1.\n            stage_level = team.stage[PkmStat.ATTACK] - opp_team.stage[PkmStat.DEFENSE]\n 

In [3]:
def estimate_damage(move_type: PkmType, pkm_type: PkmType, move_power: float, opp_pkm_type: PkmType,
                    attack_stage: int, defense_stage: int, weather: WeatherCondition) -> float:
    '''
    from the repo
    '''
    stab = 1.5 if move_type == pkm_type else 1.
    if (move_type == PkmType.WATER and weather == WeatherCondition.RAIN) or (
            move_type == PkmType.FIRE and weather == WeatherCondition.SUNNY):
        weather = 1.5
    elif (move_type == PkmType.WATER and weather == WeatherCondition.SUNNY) or (
            move_type == PkmType.FIRE and weather == WeatherCondition.RAIN):
        weather = .5
    else:
        weather = 1.
    stage_level = attack_stage - defense_stage
    stage = (stage_level + 2.) / 2 if stage_level >= 0. else 2. / (np.abs(stage_level) + 2.)
    damage = TYPE_CHART_MULTIPLIER[move_type][opp_pkm_type] * stab * weather * stage * move_power

    #if is_debug:
    multiplier = TYPE_CHART_MULTIPLIER[move_type][opp_pkm_type]
    debug_string = f'stab: {stab}, weather: {weather}, stage: {stage}, damage: {damage}, multiplier: {multiplier}, '\
        f'move_type {move_type}, move_power: {move_power}, opp_pkm_type: {opp_pkm_type}, '

    #print(damage, move_type, pkm_type, move_power, opp_pkm_type, attack_stage, defense_stage, weather)
    return damage, debug_string

#### step through env

In [6]:
team_generator = RandomTeamGenerator(2)

for i in range(10000):
    agent_team = team_generator.get_team().get_battle_team([0, 1, 2])
    opp_team = team_generator.get_team().get_battle_team([0, 1, 2])

    test_env = PkmBattleEnv((agent_team, opp_team),
                    encode=(False, False), debug=True)
    state, info = test_env.reset()

    # print("pre move")
    # team_0_active = state[0].teams[0].active
    # state_0_team_1 = state[0].teams[1]
    # active = state_0_team_1.active

    # attack into a swap
    move_int = np.random.randint(0, 4)
    attacking_pkm = state[0].teams[0].active
    #defending_pkm = state[1].teams[0].active
    defending_pkm = state[1].teams[0].party[0]

    move_type = attacking_pkm.moves[move_int].type
    defending_pkm_type = defending_pkm.type

    # if defending_pkm.max_hp > 240.:
    #     continue

    dmg_estimate, debug_string = estimate_damage(move_type,
                                   attacking_pkm.type, attacking_pkm.moves[move_int].power,
                                   defending_pkm_type , 0, 0, test_env.weather.condition)
    
    # estimate_damage(move_type: PkmType, pkm_type: PkmType, move_power: float, opp_pkm_type: PkmType,
    #                 attack_stage: int, defense_stage: int, weather: WeatherCondition) -> float:
    
    state, _, _, _, _ = test_env.step([move_int, 4])

    # see dmg taken for pkm that took damage
    dmg_taken = 0
    for pkm in [state[1].teams[0].active] + state[1].teams[0].party:
        if pkm.max_hp > pkm.hp:
            dmg_taken = pkm.max_hp - pkm.hp
            if pkm.hp == 0:
                is_fainted = True
            else:
                is_fainted = False
            break

    # dmg_taken = state[1].teams[0].active.max_hp - state[1].teams[0].active.hp

    if np.abs(dmg_estimate - dmg_taken) >= 1.:
        if not is_fainted:
            print("---")
            print("Damage values not equal and pkm not fainted")
            print(f"dmg_estimate: {dmg_estimate}, dmg_taken: {dmg_taken}, is_fainted: {is_fainted}, move_int: {move_int}, iter {i}")
            print(move_type, defending_pkm_type)
            #print(debug_string)
            #print("multiplier", TYPE_CHART_MULTIPLIER[move_type][defending_pkm_type])
            # print("move_type", move_type)
            # print("defending pkm type", defending_pkm_type)
            print("________")
            break


In [25]:
pprint.pprint(test_env.log)

('TURN 1\n'
 '\n'
 'MOVE: Trainer 1 with Pkm(Type=DRAGON, HP=228, Moves={PkmMove(Power=66, '
 'Acc=1.0, PP=10, Type=DRAGON), PkmMove(Power=174, Acc=1.0, PP=9, Type=DARK), '
 'PkmMove(Power=30, Acc=1.0, PP=10, Type=STEEL), PkmMove(Power=102, Acc=1.0, '
 'PP=10, Type=STEEL), }) uses PkmMove(Power=174, Acc=1.0, PP=9, Type=DARK)\n'
 'DAMAGE: deals 120.0 damage, hp reduces from 120.0 to 0.0 for Pkm(Type=ROCK, '
 'HP=0, Moves={PkmMove(Power=174, Acc=1.0, PP=10, Type=ROCK), '
 'PkmMove(Power=30, Acc=1.0, PP=10, Type=GROUND), PkmMove(Power=246, Acc=1.0, '
 'PP=10, Type=STEEL), PkmMove(Power=30, Acc=1.0, PP=10, Type=ICE), })\n'
 'CANNOT MOVE: Trainer 0 cannot move\n'
 'FAINTED: Pkm(Type=ROCK, HP=0, Moves={PkmMove(Power=174, Acc=1.0, PP=10, '
 'Type=ROCK), PkmMove(Power=30, Acc=1.0, PP=10, Type=GROUND), '
 'PkmMove(Power=246, Acc=1.0, PP=10, Type=STEEL), PkmMove(Power=30, Acc=1.0, '
 'PP=10, Type=ICE), })\n')


In [26]:
for pkm in [state[1].teams[0].active] + state[1].teams[0].party:
    print(pkm.type, pkm.hp, pkm.max_hp, )

PkmType.DRAGON 228.0 228.0
PkmType.DARK 120.0 120.0
PkmType.WATER 120.0 120.0


In [21]:
for pkm in [state[0].teams[0].active] + state[1].teams[0].party:
    print(pkm.type, pkm.hp, pkm.max_hp, )

PkmType.FAIRY 264.0 264.0
PkmType.DARK 120.0 120.0
PkmType.WATER 120.0 120.0


PkmType.FAIRY 120.0 120.0
PkmType.ROCK 192.0 192.0
PkmType.ICE 120.0 120.0


In [20]:
print(state[0].teams[0].active.moves[move_int].type, state[0].teams[0].active.moves[move_int].power)

PkmType.GROUND 138.0


In [23]:
for pkm in [state[1].teams[0].active] + state[1].teams[0].party:
    print(pkm.type, pkm.max_hp, pkm.hp)

PkmType.FLYING 156.0 156.0
PkmType.GHOST 120.0 120.0
PkmType.DRAGON 156.0 156.0


In [22]:
print(pkm.type, pkm.max_hp, pkm.hp)

PkmType.DRAGON 156.0 156.0


In [18]:
move_int

1

In [17]:
dmg_estimate

0.0

In [10]:
state[1].teams[0].active.hp

264.0

In [9]:
state[1].teams[0].active.max_hp

264.0

### Make a dmg to and from dict

In [ ]:

def get_turns_to_faint_list(team_1_game_state, team_2_game_state, max_turns_to_faint_value):
    '''
    put in zero for the fainted
        handle the check for fainted in HP
    assume everything is revealed
        handle masking elsewhere
    '''

    # Get weather condition
    weather = team_1_game_state.weather.condition

    # Get my Pokémon team
    team_1 = team_1_game_state.teams[0]
    team_1_pkm_list = [team_1.active] + team_1.party

    # Get opponent's team
    team_2 = team_2_game_state.teams[0]
    team_2_pkm_list = [team_2.active] + team_2.party

    # Iterate over all my Pokémon and their moves to find the most damaging move
    best_damage_list = []
    turns_to_faint_list = []
    hp_list = []

    for team_1_pkm_index, team_1_pkm in enumerate(team_1_pkm_list):
        

        for team_2_pkm_index, team_2_pkm in enumerate(team_2_pkm_list):
            # Initialize variables for the best move and its damage
            best_damage = -np.inf

            if team_1_pkm_index == 0:
                team_1_attack_stage = team_1.stage[PkmStat.ATTACK]
            else:
                team_1_attack_stage = 0
            
            if team_2_pkm_index == 0:
                team_2_defense_stage = team_2.stage[PkmStat.DEFENSE]
            else:
                team_2_defense_stage = 0

            for move_index, move in enumerate(team_1_pkm.moves):
                
                damage = estimate_damage(move.type, team_1_pkm.type, move.power, team_2_pkm.type, team_1_attack_stage,
                                            team_2_defense_stage, weather)

                # Check if the current move has higher damage than the previous best move
                if damage > best_damage:
                    best_damage = damage

            # get best dmg for each pokemon
            best_damage_list.append(best_damage)
            hp_list.append(team_2_pkm.hp)

            if best_damage > 0.:
                turns_to_faint = math.ceil(team_2_pkm.hp / best_damage)
            else:
                turns_to_faint = max_turns_to_faint_value

            turns_to_faint_list.append(turns_to_faint)

    # print(turns_to_faint_list)
    # print(best_damage_list)
    # print(hp_list)

    return turns_to_faint_list


def estimate_damage(move_type: PkmType, pkm_type: PkmType, move_power: float, opp_pkm_type: PkmType,
                    attack_stage: int, defense_stage: int, weather: WeatherCondition) -> float:
        '''
        from updated repo
        '''
        stab = 1.5 if move_type == pkm_type else 1.
        if (move_type == PkmType.WATER and weather == WeatherCondition.RAIN) or (
                move_type == PkmType.FIRE and weather == WeatherCondition.SUNNY):
            weather = 1.5
        elif (move_type == PkmType.WATER and weather == WeatherCondition.SUNNY) or (
                move_type == PkmType.FIRE and weather == WeatherCondition.RAIN):
            weather = .5
        else:
            weather = 1.
        stage_level = attack_stage - defense_stage
        stage = (stage_level + 2.) / 2 if stage_level >= 0. else 2. / (np.abs(stage_level) + 2.)
        damage = TYPE_CHART_MULTIPLIER[move_type][opp_pkm_type] * stab * weather * stage * move_power

        #print(damage, move_type, pkm_type, move_power, opp_pkm_type, attack_stage, defense_stage, weather)
        return damage


def save_object_as_pkl(object_to_save, save_tag):
    '''
    Save object a pickle file
    '''
    with open(f'{save_tag}.pickle', 'wb') as handle:
        pickle.dump(object_to_save, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
test_iters = 5000000
max_state_value = 100
time_start = time.time()

raw_stats_dict = {}
turns_to_faint_dict = {}

team_generator = RandomTeamGenerator(2)

for i in range(test_iters):
    agent_team = team_generator.get_team().get_battle_team([0, 1, ])
    opp_team = team_generator.get_team().get_battle_team([0, 1, ])

    env = PkmBattleEnv((agent_team, opp_team), encode=(False, False)) 

    game_state, info = env.reset()

    team_1_faint_list = get_turns_to_faint_list(game_state[0], game_state[1], max_state_value)
    team_2_faint_list = get_turns_to_faint_list(game_state[1], game_state[0], max_state_value)

    state_tuple = tuple(team_1_faint_list + team_2_faint_list)

    if state_tuple in raw_stats_dict:
        raw_stats_dict[state_tuple] += 1
    else:
        raw_stats_dict[state_tuple] = 1

    for turns in team_1_faint_list + team_2_faint_list:
        if turns in turns_to_faint_dict:
            turns_to_faint_dict[turns] += 1
        else:
            turns_to_faint_dict[turns] = 1

time_end = time.time()
print(f"Time to run {time_end - time_start:.3f} seconds")

print(len(raw_stats_dict))
print(len(turns_to_faint_dict))

save_object_as_pkl(raw_stats_dict, f'turns_to_faint_state_dict_{int(time_start)}')
save_object_as_pkl(turns_to_faint_dict, f'turns_to_faint_count_dict_{int(time_start)}')

In [ ]:
# so like 5-8 combined and then like 9 and over

In [ ]:
pprint.pprint(turns_to_faint_dict)

In [ ]:
# pprint.pprint(raw_stats_dict)


#### 3v3

In [ ]:
test_iters = 5000000
max_state_value = 100
time_start = time.time()

raw_stats_dict = {}
turns_to_faint_dict = {}

team_generator = RandomTeamGenerator(2)

for i in range(test_iters):
    agent_team = team_generator.get_team().get_battle_team([0, 1, 2])
    opp_team = team_generator.get_team().get_battle_team([0, 1, 2])

    env = PkmBattleEnv((agent_team, opp_team), encode=(False, False)) 

    game_state, info = env.reset()

    team_1_faint_list = get_turns_to_faint_list(game_state[0], game_state[1], max_state_value)
    team_2_faint_list = get_turns_to_faint_list(game_state[1], game_state[0], max_state_value)

    state_tuple = tuple(team_1_faint_list + team_2_faint_list)

    if state_tuple in raw_stats_dict:
        raw_stats_dict[state_tuple] += 1
    else:
        raw_stats_dict[state_tuple] = 1

    for turns in team_1_faint_list + team_2_faint_list:
        if turns in turns_to_faint_dict:
            turns_to_faint_dict[turns] += 1
        else:
            turns_to_faint_dict[turns] = 1

time_end = time.time()
print(f"Time to run {time_end - time_start:.3f} seconds")

print(len(raw_stats_dict))
print(len(turns_to_faint_dict))

save_object_as_pkl(raw_stats_dict, f'turns_to_faint_state_dict3v3_{int(time_start)}')
save_object_as_pkl(turns_to_faint_dict, f'turns_to_faint_count_dict3v3_{int(time_start)}')

In [ ]:
pprint.pprint(turns_to_faint_dict)

In [ ]:
pprint.pprint(raw_stats_dict)

In [ ]:
for k,v in raw_stats_dict.items():
    print(v, k)
    print(len(k))
    break

#### Scrap

In [ ]:


# def get_turns_to_faint_array(hp_list, best_dmg_list, max_state_value):
#     '''
#     hp list is indexed 0 to 3 with
#     0 agent active
#     1 agent party best
#     2 opp active
#     3 opp party best

#     best_dmg_list is indexed 0 to 7
#     0 agent active dmg to opp active
#     1 agent active dmg to opp party best
#     2 agent party best dmg to opp active
#     3 agent party best dmg to opp party best
#     4 opp active dmg to agent active
#     5 opp active dmg to agent party best
#     6 opp party best dmg to agent active
#     7 opp party best dmg to agent party best

#     '''
#     turns_to_faint_array = np.ones((8, ), dtype=np.float32) * -1.

#     agent_active_hp_index = 0
#     agent_party_best_hp_index = 1
#     opp_active_hp_index = 2
#     opp_party_best_hp_index = 3

#     agent_active_dmg_to_opp_active_index = 0
#     agent_active_dmg_to_opp_party_best_index = 1
#     agent_party_best_dmg_to_opp_active_index = 2
#     agent_party_best_dmg_to_opp_party_best_index = 3
#     opp_active_dmg_to_agent_active_index = 4
#     opp_active_dmg_to_agent_party_best_index = 5
#     opp_party_best_dmg_to_agent_active_index = 6
#     opp_party_best_dmg_to_agent_party_best_index = 7

#     turns_to_faint_array[0] = get_dmg_turns_to_faint(hp_list[agent_active_hp_index], 
#         best_dmg_list[agent_active_dmg_to_opp_active_index], max_state_value)
#     turns_to_faint_array[1] = get_dmg_turns_to_faint(hp_list[agent_active_hp_index],
#         best_dmg_list[agent_active_dmg_to_opp_party_best_index], max_state_value)

#     turns_to_faint_array[2] = get_dmg_turns_to_faint(hp_list[agent_party_best_hp_index],
#         best_dmg_list[agent_party_best_dmg_to_opp_active_index], max_state_value)
#     turns_to_faint_array[3] = get_dmg_turns_to_faint(hp_list[agent_party_best_hp_index],
#         best_dmg_list[agent_party_best_dmg_to_opp_party_best_index], max_state_value)

#     turns_to_faint_array[4] = get_dmg_turns_to_faint(hp_list[opp_active_hp_index],
#         best_dmg_list[opp_active_dmg_to_agent_active_index], max_state_value)
#     turns_to_faint_array[5] = get_dmg_turns_to_faint(hp_list[opp_active_hp_index],
#         best_dmg_list[opp_active_dmg_to_agent_party_best_index], max_state_value)

#     turns_to_faint_array[6] = get_dmg_turns_to_faint(hp_list[opp_party_best_hp_index],
#         best_dmg_list[opp_party_best_dmg_to_agent_active_index], max_state_value)
#     turns_to_faint_array[7] = get_dmg_turns_to_faint(hp_list[opp_party_best_hp_index],
#         best_dmg_list[opp_party_best_dmg_to_agent_party_best_index], max_state_value)

    
#     turns_to_faint_array = np.ceil(turns_to_faint_array)
#     turns_to_faint_array = turns_to_faint_array.astype(np.int32)
#     turns_to_faint_array = turns_to_faint_array.clip(-1, max_state_value)

#     return turns_to_faint_array

# turns_to_faint_array = np.ones((8, ), dtype=np.int32) * -1
# turns_to_faint_array
# np.ones((8, ), dtype=np.float32) 
# a = np.array([1.1, 2.0, 3, 4.00001, 5.00000, 5.99999, 7, 8])  
# b = np.ceil(a)
# b = b.astype(int)
# c = tuple(b)
# c
# import math
# type(math.ceil(2/1.5))

### Testing MC Env Learning in 2 v 2 environment

In [ ]:
'''
working
    finish up the loop function
        smoke test it
        see if saved action dict does anything eval wise. does it even trigger?
            are results better
    run tests on dict action space size
    probably want to expand functions to at least what opp dmg is to party
    check the functions work

to do important:
    can't swap if swp is identical or will be stuck in a swap loop
    maybe add memory saying if swapped at 2 v 2 (or other levels as well)

to do improvements:
    know the opp dmg to current team. can add those states
        DONE YES confirm this. might be a weird trick with revealed states
    maybe find how many states are possible with raw values
'''

#### Constants and loading

In [ ]:
import numpy as np
import time
import copy

from vgc.datatypes.Objects import PkmTeam, Pkm, GameState, Weather
from vgc.engine.PkmBattleEnv import PkmBattleEnv
from vgc.util.generator.PkmTeamGenerators import RandomTeamGenerator

from vgc.datatypes.Constants import TYPE_CHART_MULTIPLIER, MAX_HIT_POINTS, MOVE_MAX_PP, DEFAULT_TEAM_SIZE
from vgc.datatypes.Objects import PkmMove, Pkm, PkmTeam, GameState, Weather
from vgc.datatypes.Types import PkmStat, PkmType, WeatherCondition, \
    N_TYPES, N_STATUS, N_STATS, N_ENTRY_HAZARD, N_WEATHER, PkmStatus, PkmEntryHazard


import pprint
import pickle

In [ ]:
'''
grab the value from the lookup dict and see if can be used

DRY for the two loops

review flow of all code

reveiw and test all parts of the code

can this be generalized more?
    ie not just for the first move when eval but any move?
    idk, probably not for now

due to the variability in outcomes I think I need a ton of results to tell if swap is better or not
    like 1000 for each state and since like 500,000 states that is 500,000,000 battles

TEST need to turn the states into the dict
    TEST want to store counts and running mean of the reward

TEST set up the evaluation loop for this agent vs. the base always attack agent

TEST need to test how many battles can get through in how much time

TEST saving the action dict

TEST need to select the initial action
TEST need to then to attack later
TEST need to convert the attack action into the best attack
TEST need to convert the swap action into a swap
TEST need to store the result of the battle
TEST convert outcome into a rewards

Later

functionalize the build dict loop
functionalize the eval
can run the dict and eval in python files
can parallelize the dict building

maybe store all states in a list then to the dict
    works I think as long as all actions past that point are attacks
    could then combine that later with a swap at that point maybe?
        idk maybe not... could be the 2nd swap and then things are necessarily clear
            ie initial pkm has been revealed and may have taken dmg (though maybe that doesn't matter)

possibly store the state dict attack action for non first actions as well

            
need to add the hiding part
    i guess it's more like some states don't know opp dmg to current pkm and sometimes do

    maybe parallelize if any of this works


can I get the best dmg from both teams even if the pkm stuff is hidden and not revealed?
    probably yes since passing in the team specific state

can check to see how accurate the attack function is
'''

#### Functions for Loops

In [ ]:
def turn_agent_action_into_env_action(action, agent_game_state):
        '''
        Action values are
        0: select best move
        1: switch to first pkm
        2: switch to second pkm

        Env actions are
        0 to 3: action of active pokm
        4: switch to first pkm
        5: switch to second pkm
        '''
        # always get best move and action dmg list
        best_active_action, best_damage_list = get_best_active_damage_action(agent_game_state)

        if action == 0:
            # get best dmg action
            action = best_active_action
        else:
            # switch to first or second pkm if alive
            if action == 1 or action == 2:
                pkm = agent_game_state.teams[0].party[action-1]
                if pkm.fainted() or pkm.hp <= 0.0:
                    action = best_active_action
                else:
                    action = action + 3
            else:
                action = best_active_action

        return action, best_damage_list


def get_best_active_damage_action(g: GameState):
    '''
    '''
    # Get weather condition
    weather = g.weather.condition

    # Get my Pokémon team
    my_team = g.teams[0]
    my_pkms = [my_team.active] + my_team.party

    # Get opponent's team
    opp_team = g.teams[1]
    opp_active = opp_team.active

    opp_active_type = opp_active.type
    opp_defense_stage = opp_team.stage[PkmStat.DEFENSE]

    # Iterate over all my Pokémon and their moves to find the most damaging move
    best_dmg_list = []
    best_move_list = []

    for i, pkm in enumerate(my_pkms):
        # Initialize variables for the best move and its damage
        best_damage = -np.inf
        best_move_id = -1

        if i == 0:
            my_attack_stage = my_team.stage[PkmStat.ATTACK]
        else:
            my_attack_stage = 0

        for j, move in enumerate(pkm.moves):
            
            damage = estimate_damage(move.type, pkm.type, move.power, opp_active_type, my_attack_stage,
                                        opp_defense_stage, weather)
            
            # Check if the current move has higher damage than the previous best move
            if damage > best_damage:
                best_move_id = j + i * 4 # think for 2024 j is 0 to 3 for each
                best_damage = damage

        # get best move and dmg for each pokemon
        best_dmg_list.append(best_damage)
        best_move_list.append(best_move_id)

    active_pkm_best_move_id = best_move_list[0]

    if active_pkm_best_move_id < 0 or active_pkm_best_move_id > 3:
        print(f"Error: best move id { active_pkm_best_move_id } not in expected range")
        active_pkm_best_move_id = 0

    return active_pkm_best_move_id, best_dmg_list


def estimate_damage(move_type: PkmType, pkm_type: PkmType, move_power: float, opp_pkm_type: PkmType,
                    attack_stage: int, defense_stage: int, weather: WeatherCondition) -> float:
        '''
        Not from original code. from updated repo
        '''
        stab = 1.5 if move_type == pkm_type else 1.
        if (move_type == PkmType.WATER and weather == WeatherCondition.RAIN) or (
                move_type == PkmType.FIRE and weather == WeatherCondition.SUNNY):
            weather = 1.5
        elif (move_type == PkmType.WATER and weather == WeatherCondition.SUNNY) or (
                move_type == PkmType.FIRE and weather == WeatherCondition.RAIN):
            weather = .5
        else:
            weather = 1.
        stage_level = attack_stage - defense_stage
        stage = (stage_level + 2.) / 2 if stage_level >= 0. else 2. / (np.abs(stage_level) + 2.)
        damage = TYPE_CHART_MULTIPLIER[move_type][opp_pkm_type] * stab * weather * stage * move_power

        #print(damage, move_type, pkm_type, move_power, opp_pkm_type, attack_stage, defense_stage, weather)
        return damage


def save_object_as_pkl(object_to_save, save_tag):
    '''
    Save object a pickle file
    '''
    with open(f'{save_tag}.pickle', 'wb') as handle:
        pickle.dump(object_to_save, handle, protocol=pickle.HIGHEST_PROTOCOL)

# make a dict that has keys for 0 to 100 and values for the action dict
def make_lookup_dict():
    lookup_dict = {}
    for i in range(100):
        if i <= 40:
            lookup_value = i // 5
        else:
            lookup_value = 4 + i // 10
        lookup_dict[i] = lookup_value
    return lookup_dict

# lookup_dict = make_lookup_dict()
# pprint.pprint(lookup_dict)

def get_win_loss_reward(terminated, winner, player_index):
    '''
    Does a reward for winning or losing
    winner is -1 unless a winner has been picked
    '''
    reward = 0.
    if terminated:

        if winner == 0 or winner == 1:
            if winner == player_index:
                reward = 1.
            else:
                reward = -1.
        #print(f"reward {reward} | terminated {terminated} | winner {self.env.winner} | player_index {player_index}|")
    return reward

def get_running_mean(old_mean, old_count, new_value):
    '''
    '''
    new_mean = (old_mean * old_count + new_value) / (old_count + 1)
    
    return new_mean


def add_results_to_action_dict(action_dict, state_key, agent_first_move, win_int):
    '''
    '''
    count_key = "count"
    sum_wins_key = "sum_wins"

    if state_key in action_dict:
        if agent_first_move in action_dict[state_key]:
            action_dict[state_key][agent_first_move][sum_wins_key] += win_int
            action_dict[state_key][agent_first_move][count_key] += 1
        else:
            action_dict[state_key][agent_first_move] = {}
            action_dict[state_key][agent_first_move][sum_wins_key] = win_int
            action_dict[state_key][agent_first_move][count_key] = 1
    else:
        action_dict[state_key] = {}
        action_dict[state_key][agent_first_move] = {}
        action_dict[state_key][agent_first_move][sum_wins_key] = win_int
        action_dict[state_key][agent_first_move][count_key] = 1

def add_action_to_pkm_env_action_dict(env_action, my_dict, team_key):
    if env_action in my_dict[team_key]:
        my_dict[team_key][env_action] += 1
    else:
        my_dict[team_key][env_action] = 1

    return my_dict



# def add_results_to_action_dict(action_dict, state_key, agent_first_move, agent_reward):
#     '''
#     '''
#     if state_key in action_dict:
#         if agent_first_move in action_dict[state_key]:
#             action_dict[state_key][agent_first_move]["avg_reward"] = get_running_mean(action_dict[state_key][agent_first_move]["avg_reward"],
#                                                                                   action_dict[state_key][agent_first_move]["count"], agent_reward)
#             action_dict[state_key][agent_first_move]["count"] += 1
#         else:
#             action_dict[state_key][agent_first_move] = {}
#             action_dict[state_key][agent_first_move]["avg_reward"] = agent_reward
#             action_dict[state_key][agent_first_move]["count"] = 1

# a = (1, 2, 3)
# type(a)
# # combine two tuples
# b = a + (4, 5, 6)
# b
# # append the value 7 to the tuple
# b = b + (7,)
# b


def get_hp_array(game_state_agent, game_state_opp):
    '''
    '''
    agent_pkm_hp_list = [game_state_agent.teams[0].active.hp]

    for pkm in game_state_agent.teams[0].party:
        agent_pkm_hp_list.append(pkm.hp)

    opp_active_pkm_hp = game_state_opp.teams[0].active.hp

    hp_array = np.array(agent_pkm_hp_list + [opp_active_pkm_hp])

    return hp_array

def get_hp_list(game_state_agent, game_state_opp):
    '''
    '''
    agent_pkm_hp_list = [game_state_agent.teams[0].active.hp]

    for pkm in game_state_agent.teams[0].party:
        agent_pkm_hp_list.append(pkm.hp)

    opp_active_pkm_hp = game_state_opp.teams[0].active.hp

    hp_list = agent_pkm_hp_list + [opp_active_pkm_hp]

    return hp_list

def turn_game_state_into_dict_key(game_state_agent, game_state_opp,lookup_dict,
    dmg_array,                      
    pkm_hp_max = 480., dmg_scale_value = 600.):
    '''
    tuple is (
        # HP
        agent_active_pkm_hp, agent_party+_pkm_hp, opp_active_pkm_hp,
        # DMG to opp
        agent_active_pkm_dmg, agent_party_pkm_dmg,
        # dmg from opp
        # do this later
        )
    
    If everything is on the scale of 0 to 100 picturing
    8 buckets from 0 to 40 with increments of 5
    6 buckets from 40 to 100 with increments of 10

    preload a dict with the look up, then scal everything here

    scaling dmg more than max hp so if move can do over 480 dmg has some sort of knowledge ofit
    '''
    # get arrays to make tuples out of for dict key
    hp_array = get_hp_array(game_state_agent, game_state_opp)

    hp_tuple = scale_hp_and_get_dict_value(hp_array, pkm_hp_max, lookup_dict)
    dmg_tuple = scale_hp_and_get_dict_value(dmg_array, dmg_scale_value, lookup_dict)

    dict_key = hp_tuple + dmg_tuple

    return dict_key


def scale_hp_and_get_dict_value(hp_array, max_hp, lookup_dict):
    '''
    '''

    # scale by max hp then multiply by 100 to get into 0 to 99 range
    hp_array = (hp_array / max_hp) * 100
    # round, convert to int then clip to 0 to 99
    hp_array = hp_array.round(0).astype(int).clip(0, 99)

    

    # for hp in hp_array:
    #     hp_values_from_dict_tuple = hp_values_from_dict_tuple + (lookup_dict[hp],)

    hp_values_from_dict_tuple = tuple(lookup_dict[hp] for hp in hp_array)

    return hp_values_from_dict_tuple


    

In [ ]:
# lookup_dict = make_lookup_dict()

# a = turn_game_state_into_dict_key(game_state[0], game_state[1], lookup_dict, dmg_array)

# print(a)

# a = (1, 2, 3)
# type(a)
# # combine two tuples
# b = a + (4, 5, 6)
# b
# print(b)
# print(type(b))

In [ ]:
# team_generator = RandomTeamGenerator(2)

# agent_team = team_generator.get_team().get_battle_team([0, 1, ])
# opp_team = team_generator.get_team().get_battle_team([0, 1, ])

# # set new environment with teams
# env = PkmBattleEnv((agent_team, opp_team),
#                 encode=(False, False)) 

# game_state, info = env.reset()

# # for pkm in game_state[0].teams[1].party:
# #     print(pkm.hp, pkm.fainted())
# #     # if pkm.hp > 0.0 or not pkm.fainted():
# #     #     is_more_than_opp_pkm_alive = True
# #     #     break



##### train eval loop function

In [ ]:
def get_best_statistical_action(action_dict, state_key):
    '''
    (8, 8, 6, 5, 4) {'attack': {'sum_wins': 89, 'count': 194}, 'swap': {'sum_wins': 52, 'count': 167}}
    '''
    best_action = 0
    attack_key = 'attack'
    swap_key = 'swap'
    win_key = 'sum_wins'
    count_key = 'count'

    STOPPED HERE
    # check if state_key in action_dict
    # get the count and the wins
    # plut them into the chi squared test

    # if state_key in action_dict:
    #     best_action = max(action_dict[state_key], key=action_dict[state_key].get)
    # else:
    #     best_action = 0

    return best_action

In [ ]:
def build_train_eval_loop(num_battles, is_eval, run_tag, is_save=True, action_dict_to_copy=None):
    '''
    STOPPED HERE NEED TO WORK IN THE EVAL CODE
    then review logic then test
    '''
    if is_eval and action_dict_to_copy is not None:
        action_lookup_dict = copy.deepcopy(action_dict_to_copy)

    winner_dict = {
        0:0,
        1:0,
        -1:0
    }

    pkm_env_action_dict = {
        0:{},
        1:{},
    }

    action_state_results_dict = {}

    max_episode_steps = 250
    agent_index = 0

    lookup_dict = make_lookup_dict()
    team_generator = RandomTeamGenerator(2)

    time_int = int(time.time())
    save_tag =  f"_{run_tag}_{time_int}"
    start_time = time.time()


    for battle_idx in range(num_battles):
        
        agent_team = team_generator.get_team().get_battle_team([0, 1, ])
        opp_team = team_generator.get_team().get_battle_team([0, 1, ])

        # set new environment with teams
        env = PkmBattleEnv((agent_team, opp_team), encode=(False, False)) 

        game_state, info = env.reset()

        is_first_move = True
        agent_first_move = None
        state_key = None

        for episode_step in range(max_episode_steps):
            if is_first_move:
                if np.random.rand() < 0.5:
                    agent_pre_env_action = 0
                    agent_first_move = 'attack'
                else:
                    agent_pre_env_action = 1
                    agent_first_move = 'swap'
                is_first_move = False
            else:
                agent_pre_env_action = 0

            agent_env_action, agent_team_best_damage_list = turn_agent_action_into_env_action(agent_pre_env_action, game_state[0])
            opp_action, opp_best_damage_list = get_best_active_damage_action(game_state[1])

            if agent_pre_env_action == 1 and agent_env_action != 4:
                print("Error agent action is 1 but env action is not 4 ")
            elif agent_pre_env_action == 0:
                if (agent_env_action < 0 or agent_env_action > 3):
                    print("Error agent action is 0 but env action is not 0 to 3 ")
                elif len(agent_team_best_damage_list) == 0:
                    print("Error agent action is 0 but best damage list is empty")
                elif agent_team_best_damage_list[0] < 0:
                    print("Error agent action is 0 but best damage is negative")

            if opp_action < 0 or opp_action > 3:
                print("Error opp action is not 0 to 3")
            elif len(opp_best_damage_list) == 0:
                print("Error opp best damage list is empty")
            elif opp_best_damage_list[0] < 0:
                print("Error opp best damage is negative")
            
            # get the state key
            # only do this on the initial set up of the env
            if state_key is None:
                # for now just doing part dmg to opp
                #dmg_array = np.array(agent_best_damage_list + [opp_best_damage,])
                dmg_array = np.array(agent_team_best_damage_list)
                state_key = turn_game_state_into_dict_key(game_state[0], game_state[1], lookup_dict, dmg_array)
                if len(state_key) != 5:
                    print("Error state key is not 5 long")

                if is_eval:
                    # see if action look up dict says to do a different action
                    if state_key in action_lookup_dict:
                        # do logic for get if action is better than swap
                        STOPPED HERE
                        agent_pre_env_action = ...
                        agent_env_action = turn_agent_action_into_env_action(agent_pre_env_action, game_state[0])

                    pkm_env_action_dict = add_action_to_pkm_env_action_dict(agent_env_action, pkm_env_action_dict, 0)
                    pkm_env_action_dict = add_action_to_pkm_env_action_dict(opp_action, pkm_env_action_dict, 1)

            # enter action and step the env
            action_list = [agent_env_action, opp_action]
            game_state, _not_used_reward, terminated, truncated, info = env.step(action_list)  # for inference, we don't need reward

            if episode_step == max_episode_steps - 1:
                print('Warning: max steps reached')
                terminated = True

            if terminated:
                winner = env.winner
                if winner == agent_index:
                    win_int = 1
                else:
                    win_int = 0

                add_results_to_action_dict(action_state_results_dict, state_key, agent_first_move, win_int)

                if winner in winner_dict:
                    winner_dict[winner] += 1
                break

    end_time = time.time()
    print(f"Time to run {(end_time - start_time) / 60:.3f} minutes")
    print(f"Time to run {(end_time - start_time) / num_battles:.3f} seconds per battle")
    print(f"Time to run {((end_time - start_time) / num_battles / 60 / 60) * 1000000:.3f} hours per million battles")

    print(winner_dict)

    if is_save:
        save_object_as_pkl(action_state_results_dict , f'action_dict_{save_tag}')
        save_object_as_pkl(action_state_results_dict , f'action_dict_{winner_dict}')

    return winner_dict, action_state_results_dict, pkm_env_action_dict
    

In [ ]:
test_wd, test_ad, test_pead = build_train_eval_loop(10, is_eval=False, run_tag="test", is_save=False )
print(test_wd)
print(len(test_ad))

#### Build Dict Loop

In [ ]:
num_battles = 1000000

winner_dict = {
    0:0,
    1:0,
    -1:0
}

pkm_env_action_dict = {
    0:{},
    1:{},
}

action_state_results_dict = {}

max_episode_steps = 250
agent_index = 0

lookup_dict = make_lookup_dict()
team_generator = RandomTeamGenerator(2)

time_int = int(time.time())
save_tag =  f"_smoke_test_{time_int}"
start_time = time.time()


for battle_idx in range(num_battles):
    
    agent_team = team_generator.get_team().get_battle_team([0, 1, ])
    opp_team = team_generator.get_team().get_battle_team([0, 1, ])

    # set new environment with teams
    env = PkmBattleEnv((agent_team, opp_team),
                   encode=(False, False)) 

    game_state, info = env.reset()

    is_first_move = True
    agent_first_move = None
    state_key = None

    for episode_step in range(max_episode_steps):
        if is_first_move:
            if np.random.rand() < 0.5:
                agent_pre_env_action = 0
                agent_first_move = 'attack'
            else:
                agent_pre_env_action = 1
                agent_first_move = 'swap'
            is_first_move = False
        else:
            agent_pre_env_action = 0

        agent_env_action, agent_team_best_damage_list = turn_agent_action_into_env_action(agent_pre_env_action, game_state[0])
        opp_action, opp_best_damage_list = get_best_active_damage_action(game_state[1])

        if agent_pre_env_action == 1 and agent_env_action != 4:
            print("Error agent action is 1 but env action is not 4 ")
        elif agent_pre_env_action == 0:
            if (agent_env_action < 0 or agent_env_action > 3):
                print("Error agent action is 0 but env action is not 0 to 3 ")
            elif len(agent_team_best_damage_list) == 0:
                print("Error agent action is 0 but best damage list is empty")
            elif agent_team_best_damage_list[0] < 0:
                print("Error agent action is 0 but best damage is negative")

        if opp_action < 0 or opp_action > 3:
            print("Error opp action is not 0 to 3")
        elif len(opp_best_damage_list) == 0:
            print("Error opp best damage list is empty")
        elif opp_best_damage_list[0] < 0:
            print("Error opp best damage is negative")
        
        # get the state key
        # only do this on the initial set up of the env
        if state_key is None:
            # for now just doing part dmg to opp
            #dmg_array = np.array(agent_best_damage_list + [opp_best_damage,])
            dmg_array = np.array(agent_team_best_damage_list)
            state_key = turn_game_state_into_dict_key(game_state[0], game_state[1], lookup_dict, dmg_array)
            if len(state_key) != 5:
                print("Error state key is not 5 long")

        # enter action and step the env
        action_list = [agent_env_action, opp_action]
        game_state, _not_used_reward, terminated, truncated, info = env.step(action_list)  # for inference, we don't need reward

        if episode_step == max_episode_steps - 1:
            print('Warning: max steps reached')
            terminated = True

        if terminated:
            winner = env.winner
            if winner == agent_index:
                win_int = 1
            else:
                win_int = 0

            add_results_to_action_dict(action_state_results_dict, state_key, agent_first_move, win_int)
            break

end_time = time.time()
print(f"Time to run {(end_time - start_time) / 60:.3f} minutes")
print(f"Time to run {(end_time - start_time) / num_battles:.3f} seconds per battle")
print(f"Time to run {((end_time - start_time) / num_battles / 60 / 60) * 1000000:.3f} hours per million battles")

print(winner_dict)
# print(action_dict)


save_object_as_pkl(action_state_results_dict , 'action_dict_smoke_test')


In [ ]:
len(action_state_results_dict)

##### checking action dict results


In [ ]:
# for k, v in action_state_results_dict.items():
#     print(k, v)
#     break

# tuple is (
#     # HP
#     agent_active_pkm_hp, agent_party+_pkm_hp, opp_active_pkm_hp,
#     # DMG to opp
#     agent_active_pkm_dmg, agent_party_pkm_dmg,
#     # dmg from opp
#     # do this later
# )
    

In [ ]:
len(count_list)

In [ ]:
as_copy = copy.deepcopy(action_state_results_dict)

count_list = []

for k, v in as_copy.items():
    attack_count = v.get('attack', {}).get('count', 0)
    swap_count = v.get('swap', {}).get('count', 0)
    attack_wins = v.get('attack', {}).get('sum_wins', 0)
    swap_wins = v.get('swap', {}).get('sum_wins', 0)

    total_count = attack_count + swap_count
    count_list.append(total_count)

    if attack_count > 0:
        attack_win_percent = attack_wins / attack_count
    else:
        attack_win_percent = None

    if swap_count > 0:
        swap_win_percent = swap_wins / swap_count
    else:
        swap_win_percent = None

    if attack_win_percent is not None and swap_win_percent is not None and total_count > 100:
        if swap_win_percent - attack_win_percent > 0.05:
            print(k, np.round(attack_win_percent,3), np.round(swap_win_percent,3), attack_count, swap_count)

    
print(f"count statistics {np.mean(count_list):.3f} {np.std(count_list):.3f} {np.min(count_list)} {np.max(count_list)} {np.median(count_list)}")

# find number of counts >= x
count_array = np.array(count_list)
for x in [50, 75, 100, 150, 200]:
    print(x, count_array[count_array >= x].shape)

In [ ]:
# action_state_results_dict

In [ ]:
state_key

#### Eval Agents Loop
* based on two agents and results, see how statistically signficant the difference is

In [ ]:
num_battles = 10

winner_dict = {
    0:0,
    1:0,
    -1:0
}

pkm_env_action_dict = {
    0:{},
    1:{},
}

action_lookup_dict = copy.deepcopy(action_state_results_dict)

max_episode_steps = 250
agent_index = 0

lookup_dict = make_lookup_dict()
team_generator = RandomTeamGenerator(2)

time_int = int(time.time())
save_tag =  f"_smoke_test_winner_dict_{time_int}"
start_time = time.time()


for battle_idx in range(num_battles):
    
    agent_team = team_generator.get_team().get_battle_team([0, 1, ])
    opp_team = team_generator.get_team().get_battle_team([0, 1, ])

    # set new environment with teams
    env = PkmBattleEnv((agent_team, opp_team),
                   encode=(False, False)) 
 
    game_state, info = env.reset()

    # is_first_move = True
    # agent_first_move = None
    state_key = None

    for episode_step in range(max_episode_steps):

        agent_pre_env_action = 0
        agent_env_action, agent_team_best_damage_list = turn_agent_action_into_env_action(agent_pre_env_action, game_state[0])
        opp_action, opp_best_damage_list = get_best_active_damage_action(game_state[1])

        if opp_action < 0 or opp_action > 3:
            print("Error opp action is not 0 to 3")
        elif len(opp_best_damage_list) == 0:
            print("Error opp best damage list is empty")
        elif opp_best_damage_list[0] < 0:
            print("Error opp best damage is negative")
            
        if state_key is None:
            # for now just doing part dmg to opp
            #dmg_array = np.array(agent_best_damage_list + [opp_best_damage,])
            dmg_array = np.array(agent_team_best_damage_list)
            state_key = turn_game_state_into_dict_key(game_state[0], game_state[1], lookup_dict, dmg_array)

            # for now only override the initial action, could change this to possibly swap later as well
            if state_key in action_lookup_dict:
                agent_env_action = max(action_lookup_dict[state_key], key=action_lookup_dict[state_key].get)


        if agent_pre_env_action == 1 and agent_env_action != 4:
            print("Error agent action is 1 but env action is not 4 ")
        elif agent_pre_env_action == 0:
            if (agent_env_action < 0 or agent_env_action > 3):
                print("Error agent action is 0 but env action is not 0 to 3 ")
            elif len(agent_team_best_damage_list) == 0:
                print("Error agent action is 0 but best damage list is empty")
            elif agent_team_best_damage_list[0] < 0:
                print("Error agent action is 0 but best damage is negative")

        
   
        # enter action and step the env
        action_list = [agent_env_action, opp_action]

        pkm_env_action_dict = add_action_to_pkm_env_action_dict(agent_env_action, pkm_env_action_dict, 0)
        pkm_env_action_dict = add_action_to_pkm_env_action_dict(opp_action, pkm_env_action_dict, 1)

        #print(action_list)
        game_state, _not_used_reward, terminated, truncated, info = env.step(action_list)  # for inference, we don't need reward

        if episode_step == max_episode_steps - 1:
            print('Warning: max steps reached')
            terminated = True

        if terminated:
            winner = env.winner
            if winner == agent_index:
                win_int = 1
            else:
                win_int = 0

            if winner in winner_dict:
                winner_dict[winner] += 1

            break

end_time = time.time()
print(f"Time to run {(end_time - start_time) / 60:.3f} minutes")
print(f"Time to run {(end_time - start_time) / num_battles:.3f} seconds per battle")
print(f"Time to run {((end_time - start_time) / num_battles / 60 / 60) * 1000000:.3f} hours per million battles")

print(winner_dict)
print(pkm_env_action_dict)

save_object_as_pkl(winner_dict, 'eval_winner_dict_smoke_test')

### testing 3v3 stuff

In [12]:
fp = '3v3_results\\3v2_2v2smtest_10_1719939243_action_state_results_dict.pickle'   
# 2v2_2v2smtest_10_1719939082_action_state_results_dict.pickle'
# 3v3_2v2smtest_10_1719939244_action_state_results_dict.pickle
#3v2_2v2smtest_10_1719939243_action_state_results_dict


with open(fp, 'rb') as handle:
    test_dict = pickle.load(handle)
print(len(test_dict))
for k, v in test_dict.items():
    print(k, v)
    break
pprint.pprint(test_dict)

16
(1, -1, 1, -1, 1, -1, 1, 1, 1, -1, -1, -1, 0, 0, 0, 0, -1) {'swap_1': {(1, -1, 1, -1, 1, -1, 1, 1, 1, -1, -1, -1, 0, 0, 0, 0, -1): 1, 'count': 0, 'sum_wins': 0}}
{(1, -1, 1, -1, 1, -1, 1, 1, 1, -1, -1, -1, 0, 0, 0, 0, -1): {'swap_1': {'count': 0,
                                                                         'sum_wins': 0,
                                                                         (1, -1, 1, -1, 1, -1, 1, 1, 1, -1, -1, -1, 0, 0, 0, 0, -1): 1}},
 (1, 0, 2, 0, 1, 0, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 0): {'attack': {'count': 1,
                                                                  'sum_wins': 1}},
 (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0): {'attack': {'count': 0,
                                                                  'sum_wins': 0,
                                                                  (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0): 1}},
 (1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 2, 2, 0, 0, 1, 0, 0): {'attack': {'count': 0,
  

#### See if results significant
* should probably do this with ELO?
* these results and action files are good
* 6/29 ish update prior to moving into expanded stuff

In [65]:
fp = '2v2_state_dict_results\\2v2_test_0_1719717878_action_state_results_dict.pickle'

with open(fp, 'rb') as handle:
    action_dict = pickle.load(handle)
print(len(action_dict))
for k, v in action_dict.items():
    print(k, v)
    break

84859
(2, 1, 3, 3, 1, 1, 1, 3) {'attack': {'sum_wins': 2, 'count': 8}, 'swap': {'sum_wins': 0, 'count': 11}}


In [66]:
from scipy.stats import chi2_contingency

def get_chi_square_test_from_action_dict(
    action_dict,
    state_key,
    min_total_count=100,
    min_swap_count=50,
    min_attack_count=50,
    swap_key='swap', attack_key='attack',
    sum_wins_key='sum_wins', count_key='count',
    is_print_statistics=False):

    attack_action = 0
    swap_party_zero_action = 1
    swap_party_one_action = 2

    is_use_p_value = False
    is_swap_better = False
    p_value = None
    swap_win_rate_better_rate = 0.
    recommended_action = attack_action

    try:
        if state_key in action_dict:

            if swap_key in action_dict[state_key] and attack_key in action_dict[state_key]:
                swap_wins = action_dict[state_key][swap_key][sum_wins_key]
                swap_count = action_dict[state_key][swap_key][count_key]
                attack_wins = action_dict[state_key][attack_key][sum_wins_key]
                attack_count = action_dict[state_key][attack_key][count_key]

                total_count = swap_count + attack_count

                if total_count > min_total_count and swap_count > min_swap_count and attack_count > min_attack_count:

                    swap_win_percent = swap_wins / swap_count
                    attack_win_percent = attack_wins / attack_count
                    
                    if swap_win_percent > attack_win_percent:
                        is_swap_better = True
                        swap_win_rate_better_rate = swap_win_percent - attack_win_percent
                    else:
                        is_swap_better = False
                        swap_win_rate_better_rate = 0.

                    # chi squared table breaks down if any 0 values
                    # really should not have less than 5
                    if attack_wins == attack_count:
                        recommended_action = attack_action
                        # choose attack as attack always wins
                        if is_print_statistics:
                            print("Attack always wins")
                            print(f"Swap win rate: {swap_wins / swap_count:.3f} | Count {swap_count}")
                            print(f"Attack win rate: {attack_wins / attack_count:.3f} | Count {attack_count}")
                    elif swap_wins == swap_count:
                        # choose swap
                        is_use_p_value = True
                        is_swap_better = True
                        p_value = 0.
                        recommended_action = swap_party_zero_action
                        if is_print_statistics:
                            print("swap always wins, choosing swap")
                            print(f"Swap win rate: {swap_wins / swap_count:.3f} | Count {swap_count}")
                            print(f"Attack win rate: {attack_wins / attack_count:.3f} | Count {attack_count}")
                    elif swap_wins == 0:
                        recommended_action = attack_action
                        # swap always loses
                        if is_print_statistics:
                            print("Swap always loses")
                            print(f"Swap win rate: {swap_wins / swap_count:.3f} | Count {swap_count}")
                            print(f"Attack win rate: {attack_wins / attack_count:.3f} | Count {attack_count}")
                    elif attack_wins == 0:
                        # attack always loses and swap won at least once so choose swap
                        is_use_p_value = True
                        is_swap_better = True
                        p_value = 0.
                        recommended_action = swap_party_zero_action
                        if is_print_statistics:
                            print("Attack always loses, choosing swap ")
                            print(f"Swap win rate: {swap_wins / swap_count:.3f} | Count {swap_count}")
                            print(f"Attack win rate: {attack_wins / attack_count:.3f} | Count {attack_count}")
                    else:
                        contingency_table = [[swap_wins, swap_count - swap_wins], [attack_wins, attack_count - attack_wins]]
                        chi2, p_value, dof, expected = chi2_contingency(contingency_table)
                        is_use_p_value = True

                        if is_swap_better:
                            if p_value < 0.25:
                                recommended_action = swap_party_zero_action
                            elif swap_win_rate_better_rate >= .1:
                                recommended_action = swap_party_zero_action
                            elif swap_win_rate_better_rate >= .05 and p_value < .6:
                                recommended_action = swap_party_zero_action

                        if is_print_statistics:
                            #print(f'Swap Win : { win_loss_draw1[0] / sum(win_loss_draw1):.3f}')
                            print(f"Swap win rate: {swap_wins / swap_count:.3f} | Count {swap_count}")
                            print(f"Attack win rate: {attack_wins / attack_count:.3f} | Count {attack_count}")
                            print(f'Chi-square statistic: {chi2:.3f}')
                            print(f'P-value: {p_value:.5f}')

        else:
            is_use_p_value = False
            is_swap_better = False
            p_value = None
            swap_win_rate_better_rate = 0.
            recommended_action = attack_action
    except Exception as e:
        print("Error: in chi square test ", str(e) )
        is_use_p_value = False
        is_swap_better = False
        p_value = None
        swap_win_rate_better_rate = 0.
        recommended_action = attack_action
    
    return recommended_action, swap_win_rate_better_rate, is_use_p_value, is_swap_better, p_value

##### tune p value

In [67]:
num_iters = -1
num_swaps_better = 0
# num_swaps_better_max = 5
p_value_list = []
min_p_value = 1.

# testing function
def is_swap_allowed_symmetric_state(current_state, num_agent_pkm, num_opp_pkm):
    '''
    Do not allow swaps if the result would mean the same state
    '''

    if num_agent_pkm == 2 and num_opp_pkm == 2:
        # 8 keys
        # agent active to opp active
        # agent active to opp party
        # agent party to opp active
        # agent party to opp party
        # opp active to agent active
        # opp active to agent party
        # opp party to agent active
        # opp party to agent party
        # if a swap occurs
        #   index 0 becomes index 2 and index 1 becomes index 3
        #   index 4 becomes index 5 and index 6 becomes index 7

        if current_state[0] == current_state[2] and current_state[1] == current_state[3] \
            and current_state[4] == current_state[5] and current_state[6] == current_state[7]:
            print("Testing: same state, swap not allowed")
            return False
        else:
            return True

    return True
    
for test_key, _ in action_dict.items():
    num_iters += 1
    recommended_action, swap_win_rate_better_rate, is_use_p_value, is_swap_better, p_value = get_chi_square_test_from_action_dict(
        action_dict, test_key, is_print_statistics=False)

    if recommended_action == 1:
        num_swaps_better += 1
        if p_value > .5:
            print(f"--- {test_key} ---")
            recommended_action, swap_win_rate_better_rate, is_use_p_value, is_swap_better, p_value = get_chi_square_test_from_action_dict(
                action_dict, test_key, is_print_statistics=True)
            print("______")
    # if is_swap_better and p < min_p_value:
    #     p_value_list.append(p)
    #     num_swaps_better += 1

    #     if p >= .9:
    #         print(f"--- {test_key} ---")
    #         is_use_p, is_swap_better, p = get_chi_square_test_from_action_dict(action_dict, test_key, is_print_statistics=True)
    #         print("______")

print(len(action_dict), num_swaps_better)


--- (1, 1, 1, 3, 1, 1, 3, 1) ---
Swap win rate: 0.591 | Count 66
Attack win rate: 0.529 | Count 68
Chi-square statistic: 0.295
P-value: 0.58720
______
--- (2, 1, 2, 3, 2, 1, 2, 1) ---
Swap win rate: 0.321 | Count 53
Attack win rate: 0.246 | Count 57
Chi-square statistic: 0.440
P-value: 0.50719
______
--- (1, 2, 1, 4, 1, 2, 2, 3) ---
Swap win rate: 0.596 | Count 99
Attack win rate: 0.543 | Count 105
Chi-square statistic: 0.389
P-value: 0.53266
______
--- (3, 1, 3, 1, 3, 1, 1, 1) ---
Swap win rate: 0.576 | Count 59
Attack win rate: 0.500 | Count 52
Chi-square statistic: 0.377
P-value: 0.53936
______
--- (1, 3, 1, 2, 1, 2, 3, 2) ---
Swap win rate: 0.672 | Count 64
Attack win rate: 0.613 | Count 75
Chi-square statistic: 0.291
P-value: 0.58952
______
84859 438


In [61]:
print(test_key, min(test_key))

(5, 1, 2, 1, 2, 2, 2, 5) 1


In [63]:
# testing if unkowns in the state key
num_iters = -1
num_swaps_better = 0
# num_swaps_better_max = 5
p_value_list = []
min_p_value = 1.

for test_key, _ in action_dict.items():
    num_iters += 1
    recommended_action, swap_win_rate_better_rate, is_use_p_value, is_swap_better, p_value = get_chi_square_test_from_action_dict(
        action_dict, test_key, is_print_statistics=False)

    if recommended_action == 1:
        if min(test_key) == -1:
            num_swaps_better += 1
            print(f"--- {test_key} ---")
            recommended_action, swap_win_rate_better_rate, is_use_p_value, is_swap_better, p_value = get_chi_square_test_from_action_dict(
                action_dict, test_key, is_print_statistics=True)
            print("______")

        # # any unkowns in the state
        # num_swaps_better += 1
        # if p_value > .5:
        #     print(f"--- {test_key} ---")
        #     recommended_action, swap_win_rate_better_rate, is_use_p_value, is_swap_better, p_value = get_chi_square_test_from_action_dict(
        #         action_dict, test_key, is_print_statistics=True)
        #     print("______")
    # if is_swap_better and p < min_p_value:
    #     p_value_list.append(p)
    #     num_swaps_better += 1

    #     if p >= .9:
    #         print(f"--- {test_key} ---")
    #         is_use_p, is_swap_better, p = get_chi_square_test_from_action_dict(action_dict, test_key, is_print_statistics=True)
    #         print("______")

print("checking unknown states", len(action_dict), num_swaps_better)

--- (1, 1, 1, 2, -1, -1, 2, 1) ---
Swap win rate: 0.683 | Count 142
Attack win rate: 0.545 | Count 165
Chi-square statistic: 5.509
P-value: 0.01892
______
--- (1, 1, 1, 1, 1, 3, -1, -1) ---
Swap win rate: 0.847 | Count 59
Attack win rate: 0.755 | Count 151
Chi-square statistic: 1.615
P-value: 0.20375
______
--- (1, 1, 2, 2, -1, -1, 2, 1) ---
Swap win rate: 0.667 | Count 54
Attack win rate: 0.541 | Count 61
Chi-square statistic: 1.398
P-value: 0.23707
______
--- (2, 1, 2, 2, 1, 2, -1, -1) ---
Swap win rate: 0.183 | Count 71
Attack win rate: 0.092 | Count 109
Chi-square statistic: 2.452
P-value: 0.11737
______
--- (1, 1, 1, 2, 1, 2, -1, -1) ---
Swap win rate: 0.461 | Count 219
Attack win rate: 0.400 | Count 477
Chi-square statistic: 2.033
P-value: 0.15389
______
--- (2, 1, 2, 2, -1, -1, -1, -1) ---
Swap win rate: 0.355 | Count 124
Attack win rate: 0.278 | Count 176
Chi-square statistic: 1.645
P-value: 0.19958
______
--- (2, 1, 2, 2, -1, -1, 1, 1) ---
Swap win rate: 0.265 | Count 83
Attac

In [68]:
# testing function
def is_swap_allowed_symmetric_state(current_state, num_agent_pkm, num_opp_pkm):
    '''
    Do not allow swaps if the result would mean the same state
    '''

    if num_agent_pkm == 2 and num_opp_pkm == 2:
        # 8 keys
        # agent active to opp active
        # agent active to opp party
        # agent party to opp active
        # agent party to opp party
        # opp active to agent active
        # opp active to agent party
        # opp party to agent active
        # opp party to agent party
        # if a swap occurs
        #   index 0 becomes index 2 and index 1 becomes index 3
        #   index 4 becomes index 5 and index 6 becomes index 7

        if current_state[0] == current_state[2] and current_state[1] == current_state[3] \
            and current_state[4] == current_state[5] and current_state[6] == current_state[7]:
            print("Testing: same state, swap not allowed")
            return False
        else:
            return True

    return True

num_iters = -1
num_swaps_better = 0
# num_swaps_better_max = 5
p_value_list = []
min_p_value = 1.

for test_key, _ in action_dict.items():
    num_iters += 1
    recommended_action, swap_win_rate_better_rate, is_use_p_value, is_swap_better, p_value = get_chi_square_test_from_action_dict(
        action_dict, test_key, is_print_statistics=False)

    if recommended_action == 1:
        num_swaps_better += 1

        if not is_swap_allowed_symmetric_state(test_key, 2, 2):
            print("Error,")
            print(f"--- {test_key} ---")
            recommended_action, swap_win_rate_better_rate, is_use_p_value, is_swap_better, p_value = get_chi_square_test_from_action_dict(
                action_dict, test_key, is_print_statistics=True)
            print("______")
        # if p_value > .5:
        #     print(f"--- {test_key} ---")
        #     recommended_action, swap_win_rate_better_rate, is_use_p_value, is_swap_better, p_value = get_chi_square_test_from_action_dict(
        #         action_dict, test_key, is_print_statistics=True)
        #     print("______")
    # if is_swap_better and p < min_p_value:
    #     p_value_list.append(p)
    #     num_swaps_better += 1

    #     if p >= .9:
    #         print(f"--- {test_key} ---")
    #         is_use_p, is_swap_better, p = get_chi_square_test_from_action_dict(action_dict, test_key, is_print_statistics=True)
    #         print("______")

print(len(action_dict), num_swaps_better)

Testing: same state, swap not allowed
Error,
--- (2, 2, 2, 2, 1, 1, 1, 1) ---
Attack always loses, choosing swap 
Swap win rate: 0.002 | Count 2865
Attack win rate: 0.000 | Count 5921
______
Testing: same state, swap not allowed
Error,
--- (3, 1, 3, 1, 1, 1, 1, 1) ---
Attack always loses, choosing swap 
Swap win rate: 0.003 | Count 337
Attack win rate: 0.000 | Count 445
______
Testing: same state, swap not allowed
Error,
--- (3, 1, 3, 1, -1, -1, 1, 1) ---
Swap win rate: 0.395 | Count 86
Attack win rate: 0.269 | Count 104
Chi-square statistic: 2.856
P-value: 0.09101
______
84859 438


In [31]:
# test_key_list = [tuple([x]*8) for x in range(10)]
# test_key_list

test_key_list = [tuple([x]*8) for x in range(5)]

for test_key in test_key_list:
    print(f"--- {test_key} ---")
    is_use_p, is_swap_better, p = get_chi_square_test_from_action_dict(action_dict, test_key, is_print_statistics=True)
    print(is_use_p, p)
    print("______")

--- (0, 0, 0, 0, 0, 0, 0, 0) ---
False None
______
--- (1, 1, 1, 1, 1, 1, 1, 1) ---
Swap win rate: 0.285 | Count 56373
Attack win rate: 0.503 | Count 89648
Chi-square statistic: 6768.218
P-value: 0.00000
True 0.0
______
--- (2, 2, 2, 2, 2, 2, 2, 2) ---
Swap win rate: 0.175 | Count 371
Attack win rate: 0.468 | Count 453
Chi-square statistic: 77.044
P-value: 0.00000
True 1.6719395982814362e-18
______
--- (3, 3, 3, 3, 3, 3, 3, 3) ---
False None
______
--- (4, 4, 4, 4, 4, 4, 4, 4) ---
False None
______


In [32]:
num_iters = -1
num_swaps_better = 0
num_swaps_better_max = 5

for test_key, _ in action_dict.items():
    num_iters += 1
    is_use_p, is_swap_better, p = get_chi_square_test_from_action_dict(action_dict, test_key, is_print_statistics=False)

    if is_swap_better:
        print(f"--- {test_key} ---")
        is_use_p, is_swap_better, p = get_chi_square_test_from_action_dict(action_dict, test_key, is_print_statistics=True)
        print(is_use_p, p)
        print("______")
        num_swaps_better += 1
        if num_swaps_better >= num_swaps_better_max:
            break

        

--- (1, 1, 1, 2, -1, -1, 2, 1) ---
Swap win rate: 0.683 | Count 142
Attack win rate: 0.545 | Count 165
Chi-square statistic: 5.509
P-value: 0.01892
True 0.018922771494133066
______
--- (2, 2, 1, 2, 2, 2, 2, 1) ---
Swap win rate: 0.494 | Count 156
Attack win rate: 0.095 | Count 242
Chi-square statistic: 77.984
P-value: 0.00000
True 1.038903566590097e-18
______
--- (1, 2, 1, 2, 1, 2, 1, 1) ---
Swap win rate: 0.138 | Count 812
Attack win rate: 0.130 | Count 3198
Chi-square statistic: 0.259
P-value: 0.61095
True 0.6109535241152642
______
--- (1, 1, 1, 2, 1, 1, 1, 1) ---
Swap win rate: 0.267 | Count 9139
Attack win rate: 0.254 | Count 13881
Chi-square statistic: 4.629
P-value: 0.03144
True 0.03144015349882742
______
--- (2, 2, 2, 3, 1, 2, 2, 2) ---
Swap win rate: 0.186 | Count 59
Attack win rate: 0.067 | Count 75
Chi-square statistic: 3.438
P-value: 0.06370
True 0.0636988410853709
______


In [ ]:
### fucking awesome. some of these make a lot of sense

In [36]:
action_dict[test_key]

{'swap': {'sum_wins': 57, 'count': 57},
 'attack': {'sum_wins': 93, 'count': 93}}

ValueError: The internally computed table of expected frequencies has a zero element at (0, 1).

In [ ]:
from scipy.stats import chi2_contingency

def chi_square_test(win_loss_draw1, win_loss_draw2):
    """
    Performs a Chi-square test on two entities with win, loss, and draw counts.
    
    Parameters:
    - win_loss_draw1: A list or tuple of win, loss, and draw counts for entity 1 (e.g., [wins, losses, draws]).
    - win_loss_draw2: A list or tuple of win, loss, and draw counts for entity 2.
    
    Returns:
    - Chi-square statistic, p-value, and interpretation as a string.
    """
    # Create a contingency table
    contingency_table = [win_loss_draw1, win_loss_draw2]
    
    # Perform the Chi-square test
    chi2, pval, dof, expected = chi2_contingency(contingency_table)
    
    # Interpret the results
    interpretation = ("There is a statistically significant difference in outcomes between the two entities."
                      if pval < 0.05 else
                      "There is no statistically significant difference in outcomes between the two entities.")
    
    print(f'Player 1 Win rate: { win_loss_draw1[0] / sum(win_loss_draw1):.3f}')


    print(f'Chi-square statistic: {chi2:.3f}')
    print(f'P-value: {pval:.5f}')

    return chi2, pval, interpretation

# # Example usage
#chi2, pval, interpretation = chi_square_test([120, 80, 50], [130, 70, 60])

# chi2, pval, interpretation = chi_square_test([winner_dict[0],
#                                               winner_dict[1],
#                                               #winner_dict[-1]
#                                               ],
#                                              [winner_dict[1],
#                                               winner_dict[0],
#                                               #winner_dict[-1]
#                                               ],)

In [ ]:
# chi2, pval, interpretation = chi_square_test([winner_dict[0],
#                                               winner_dict[1],
#                                               #winner_dict[-1]
#                                               ],
#                                              [winner_dict[1],
#                                               winner_dict[0],
#                                               #winner_dict[-1]
#                                               ],)

In [ ]:
# test where it is significant

for x in range(150, 200, 5):
    winner_dict = {
        0:x,
        1:300-x,
        -1:0
    }
    print("--- x is ", x)
    chi2, pval, interpretation = chi_square_test([winner_dict[0],
                                                winner_dict[1],
                                                #winner_dict[-1]
                                                ],
                                                [winner_dict[1],
                                                winner_dict[0],
                                                #winner_dict[-1]
                                                ],)
    print("________")

In [ ]:
from scipy.stats import ttest_ind


def compare_rates(successes_group1, observations_group1, successes_group2, observations_group2):
    """
    Compares two observed rates using a two-sample t-test and prints the t-statistic and p-value.
    
    Parameters:
    - successes_group1: Number of successes in group 1
    - observations_group1: Number of observations in group 1
    - successes_group2: Number of successes in group 2
    - observations_group2: Number of observations in group 2
    """
    # Calculate rates
    rate_group1 = successes_group1 / observations_group1
    rate_group2 = successes_group2 / observations_group2

    # Convert rates to "success" arrays
    data_group1 = np.array([1] * successes_group1 + [0] * (observations_group1 - successes_group1))
    data_group2 = np.array([1] * successes_group2 + [0] * (observations_group2 - successes_group2))

    # Perform the two-sample t-test
    stat, pval = ttest_ind(data_group1, data_group2)

    # Print rounded t-statistic and p-value
    print("Rates: {:.3f} vs. {:.3f}".format(rate_group1, rate_group2))
    print(f'T-statistic: {stat:.5f}')
    print(f'P-value: {pval:.5f}')

# Example usage
#compare_rates(45, 100, 50, 120)

compare_rates(winner_dict[0], num_battles, winner_dict[1], num_battles)

In [ ]:
for test_wins in [150, 160, 170, 175, 200, 225, 250, 275]:
    print(test_wins)
    compare_rates(test_wins, num_battles, num_battles-test_wins, num_battles)

In [ ]:
print(episode_step)

In [ ]:
env.step(action_list) 

In [ ]:
.266 / 60 * 1000000 /60

### Testing dict size with raw values
* so with 5m iters, like 4.7m different dict values
* so some grouping needs to be done

In [ ]:
test_iters = 5000000

time_start = time.time()

raw_stats_dict = {}
team_generator = RandomTeamGenerator(2)

for i in range(test_iters):
    agent_team = team_generator.get_team().get_battle_team([0, 1, ])
    opp_team = team_generator.get_team().get_battle_team([0, 1, ])

    game_state, info = env.reset()

    env = PkmBattleEnv((agent_team, opp_team), encode=(False, False)) 

    game_state, info = env.reset()

    _agent_env_action, agent_team_best_damage_list = turn_agent_action_into_env_action(0, game_state[0])
    _opp_action, opp_best_damage_list = get_best_active_damage_action(game_state[1])

    hp_list = get_hp_list(game_state[0], game_state[1])
    #print(hp_list, agent_team_best_damage_list, opp_best_damage_list)

    state_tuple = tuple(hp_list + agent_team_best_damage_list + opp_best_damage_list)
    #print(state_tuple)

    if state_tuple in raw_stats_dict:
        raw_stats_dict[state_tuple] += 1
    else:
        raw_stats_dict[state_tuple] = 1

time_end = time.time()
print(f"Time to run {time_end - time_start:.3f} seconds")

print(len(raw_stats_dict))

save_object_as_pkl(raw_stats_dict, 'raw_state_dict_smoke_test')


In [ ]:
10956.973 /60 /60

In [ ]:
raw_hp_act_dict = copy.deepcopy(raw_stats_dict)

In [ ]:
# see the distribution of HP values and dmg values

In [ ]:
for k, v in raw_hp_act_dict.items():
    print(k,v)
    break

In [ ]:
# first 3 are HP, next four are dmg
hp_dict_count = {}
dmg_dict_count = {}
hp_list = []
dmg_list = []

for k, _ in raw_hp_act_dict.items():
    hp_key = k[:3]
    dmg_key = k[3:]

    for hp_value in hp_key:
        if hp_value in hp_dict_count:
            hp_dict_count[hp_value] += 1
        else:
            hp_dict_count[hp_value] = 1
        hp_list.append(hp_value)

    for dmg_value in dmg_key:
        if dmg_value in dmg_dict_count:
            dmg_dict_count[dmg_value] += 1
        else:
            dmg_dict_count[dmg_value] = 1
        dmg_list.append(dmg_value)
    

print(hp_dict_count)
print(dmg_dict_count)

##### notes
* so very few HP values
* also not that many low dmg values

In [70]:
pprint.pprint({336.0: 484424, 120.0: 3779517, 156.0: 3378332, 264.0: 1144966, 192.0: 2453402, 228.0: 1692512, 408.0: 164303, 300.0: 758680, 372.0: 296296, 480.0: 22651, 444.0: 75229})

{120.0: 3779517,
 156.0: 3378332,
 192.0: 2453402,
 228.0: 1692512,
 264.0: 1144966,
 300.0: 758680,
 336.0: 484424,
 372.0: 296296,
 408.0: 164303,
 444.0: 75229,
 480.0: 22651}


In [ ]:
pprint.pprint(hp_dict_count)

In [ ]:
'''
potential HP dict keys
400 and over
320 to 400
240 to 320
160 to 240
80 to 160
0 to 80


dmg values
0 to 80
80 to 160
160 to 240
240 to 320
320 to 400
400 to 480
480 to 560
560 +


5x5 would be like
0 to 96
96 to 192
192 to 288
288 to 384
384 to 480

then for dmg values could do like
0 to 120
120 to 240
240 to 360
360 to 480
480 +

'''

In [ ]:
480 / 5

In [ ]:
print(6**3 * 8**4)
print(6*6*6*8*8*8*8)

print(5**4 * 5**6)

print(4**4 * 4**6)

In [ ]:


for a in range(3,9):
    for b in range(3,10):
        print(a, b, (a**3)*(b**4))

In [ ]:
pprint.pprint(dmg_dict_count)

In [ ]:
print(len(hp_dict_count), len(dmg_dict_count))
print(11*11*11 * 67*67*67*67)

In [ ]:
print(11**6 + 67**18 )

In [ ]:
# can then put into pandas and see the distribution of values